In [11]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
import time
import os
import math
import numpy as np
import pyke as pk
import pandas as pd
import k2flix
import matplotlib.pyplot as plt

def collect(oid_list,test):
    tpfs, flux, flares = dict(), dict(), dict()
    for oid in oid_list:

        flares[oid] = pd.read_csv('results/{}/{}_flares.csv'.format(test,oid),
                                 usecols=['istart','istop','istart_no_sys_no_thruster','istop_no_sys_no_thruster'])
        flux[oid] = pd.read_csv('results/{}/{}_tpf_times_FLUX.csv'.format(test,oid))#,
                                #usecols=['flux_gap','time','flux_model'])
    return flux, flares

os.chdir('/home/ekaterina/Documents/appaloosa/stars_shortlist/M67')
oid_list = pd.read_csv('M67_full.txt',names=['EPIC'])
oid_list = oid_list.EPIC.tolist()
test = 'run_01'
flux, flares = collect(oid_list, test)

- flare number before/after cleaning
- coincidence with flags for cleaned flares
- binomial distribution vs. real distribution


In [14]:
l1, l2 = [], []
for oid in oid_list:

    l1.append(len(flares[oid].istart.dropna()))
    l2.append(len(flares[oid].istart_no_sys_no_thruster.dropna()))

c_flares = pd.DataFrame({'before':l1,'after':l2},index=oid_list)
c_flares['difference'] = c_flares.before - c_flares.after
c_flares.describe()

,after,before,difference
count,75.000000,75.000000,75.000000
mean,20.746667,37.653333,16.906667
std,16.969841,26.066699,10.571728
min,3.000000,11.000000,3.000000
25%,11.000000,23.000000,10.000000
50%,15.000000,28.000000,14.000000
75%,22.000000,38.000000,19.500000
max,82.000000,129.000000,58.000000
